In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Retinoblastoma"
cohort = "GSE29686"

# Input paths
in_trait_dir = "../../input/GEO/Retinoblastoma"
in_cohort_dir = "../../input/GEO/Retinoblastoma/GSE29686"

# Output paths
out_data_file = "../../output/preprocess/Retinoblastoma/GSE29686.csv"
out_gene_data_file = "../../output/preprocess/Retinoblastoma/gene_data/GSE29686.csv"
out_clinical_data_file = "../../output/preprocess/Retinoblastoma/clinical_data/GSE29686.csv"
json_path = "../../output/preprocess/Retinoblastoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Coexpression of Normally Incompatible Developmental Pathways in Retinoblastoma Genesis"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['genotype/variation: TKO', 'genotype/variation: p107_single', 'genotype/variation: MDMX', 'genotype/variation: p130_DKO', 'genotype/variation: p107_DKO', 'genotype/variation: p130_TKO']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to consist of various retinoblastoma cell types
# This includes cell lines, primary tumors, xenograft-passaged cells, etc. which likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics, we can see:
# - Row 0 contains cell type information that can be mapped to trait (retinoblastoma status)
# - Age information is not available
# - Gender information is not available
trait_row = 0
age_row = None
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """
    Convert the cell type information to a binary trait value.
    1 = retinoblastoma (cell lines, primary tumor, xenograft, single tumor cell)
    0 = normal cell (not present in this dataset)
    """
    if value is None or ':' not in value:
        return None
    
    # Extract the value after the colon and strip whitespace
    value = value.split(':', 1)[1].strip().lower()
    
    # All samples appear to be retinoblastoma-related
    # Primary tumor, cell lines, and xenografts are all considered positive for retinoblastoma
    if 'cell line' in value or 'primary tumor' in value or 'xenograft' in value or 'tumor cell' in value:
        return 1
    else:
        # If there were normal samples, they would be coded as 0, but there don't appear to be any
        return None

def convert_age(value):
    """Placeholder function for age conversion (not used in this dataset)"""
    return None

def convert_gender(value):
    """Placeholder function for gender conversion (not used in this dataset)"""
    return None

# 3. Save Metadata
# trait_row is not None, so trait data is available
is_trait_available = trait_row is not None
# Use validate_and_save_cohort_info for initial filtering
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical feature extraction
if trait_row is not None:
    # Extract clinical features using the provided function
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    print("Preview of extracted clinical features:")
    print(preview_df(clinical_df))
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of extracted clinical features:
{'GSM736310': [nan], 'GSM736311': [nan], 'GSM736312': [nan], 'GSM736313': [nan], 'GSM736314': [nan], 'GSM736315': [nan], 'GSM736316': [nan], 'GSM736317': [nan], 'GSM736318': [nan], 'GSM736319': [nan], 'GSM736320': [nan], 'GSM736321': [nan], 'GSM736322': [nan], 'GSM736323': [nan], 'GSM736324': [nan], 'GSM736325': [nan], 'GSM736326': [nan], 'GSM736327': [nan], 'GSM736328': [nan], 'GSM736329': [nan], 'GSM736330': [nan], 'GSM736331': [nan], 'GSM736332': [nan], 'GSM736333': [nan], 'GSM736334': [nan], 'GSM736335': [nan], 'GSM736336': [nan], 'GSM736337': [nan], 'GSM736338': [nan], 'GSM736339': [nan], 'GSM736340': [nan], 'GSM736341': [nan], 'GSM736342': [nan], 'GSM736343': [nan], 'GSM736344': [nan], 'GSM736345': [nan], 'GSM736346': [nan], 'GSM736347': [nan], 'GSM736348': [nan], 'GSM736349': [nan], 'GSM736350': [nan], 'GSM736351': [nan], 'GSM736352': [nan], 'GSM736353': [nan], 'GSM736354': [nan], 'GSM736355': [nan], 'GSM736356': [nan], 'GSM736357': [nan],

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print(gene_data.index[:20])


Index(['1415670_at', '1415671_at', '1415672_at', '1415673_at', '1415674_a_at',
       '1415675_at', '1415676_a_at', '1415677_at', '1415678_at', '1415679_at',
       '1415680_at', '1415681_at', '1415682_at', '1415683_at', '1415684_at',
       '1415685_at', '1415686_at', '1415687_a_at', '1415688_at',
       '1415689_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
"""
Based on the gene identifiers shown in the output, these appear to be Affymetrix probe IDs 
(like '1007_s_at', '1053_at', etc.) rather than standard human gene symbols.

Affymetrix probe IDs are specific to microarray platforms and need to be mapped to 
standard gene symbols for consistent analysis across different platforms and studies.

Human gene symbols typically follow naming conventions like HGNC identifiers 
(e.g., BRCA1, TP53, etc.), while these are clearly platform-specific identifiers.
"""

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the gene annotation and gene expression data, determine the mapping columns
# From the preview, we can see that:
# - 'ID' corresponds to probe identifiers (e.g., '1007_s_at') which match the gene expression data
# - 'Gene Symbol' corresponds to the human gene symbols (e.g., 'DDR1 /// MIR4640')

# 2. Extract the mapping between probe IDs and gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Preview the mapped gene data
print("Gene data after mapping (first 5 genes):")
print(gene_data.head(5))


Gene data after mapping (first 5 genes):
            GSM736310  GSM736311  GSM736312  GSM736313  GSM736314  GSM736315  \
Gene                                                                           
A130033P14    5.29477    5.75712    5.50559    5.40122    5.50805    5.52647   
A730034C02   22.64450   21.06850   21.58328   21.18967   21.21202   21.69042   
A830091E24    6.72050    7.82499    8.17858    8.16945    8.52941    7.88528   
AA386476      8.02869    8.45463    7.53051    7.50578    7.52873    7.71510   
AA388235     14.56157   14.59023   14.91173   14.99273   13.90034   14.56070   

            GSM736316  GSM736317  GSM736318  GSM736319  ...  GSM736432  \
Gene                                                    ...              
A130033P14    5.54293    5.29056    5.29585    5.47990  ...    5.59210   
A730034C02   21.32234   20.88001   20.43590   22.43317  ...   22.46651   
A830091E24    8.10443    7.97413    8.43196    8.55281  ...    8.56259   
AA386476      7.40082    7.6

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Load the clinical features from the saved file
clinical_file_path = out_clinical_data_file
if os.path.exists(clinical_file_path):
    clinical_features = pd.read_csv(clinical_file_path, index_col=0)
    print(f"Clinical features loaded from {clinical_file_path}")
    print(f"Clinical features shape: {clinical_features.shape}")
else:
    # If file doesn't exist, we need to extract it again
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    print(f"Clinical features re-extracted")
    print(f"Clinical features shape: {clinical_features.shape}")

# 2. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(f"First few columns: {list(linked_data.columns[:5])}")

# 3. Handle missing values in the linked data
trait_column = trait
if trait_column not in linked_data.columns:
    # Use the first column as trait if trait name is not found
    trait_column = linked_data.columns[0]
print(f"Using trait column: {trait_column}")

# Check data types
print(f"Data type of trait column: {linked_data[trait_column].dtype}")
if linked_data[trait_column].dtype == 'object':
    try:
        linked_data[trait_column] = pd.to_numeric(linked_data[trait_column])
        print("Converted trait column to numeric type")
    except:
        print("Could not convert trait column to numeric type. Check the data format.")

linked_data_processed = handle_missing_values(linked_data, trait_column)
print(f"Shape after handling missing values: {linked_data_processed.shape}")

# 4. Determine whether the trait and demographic features are severely biased
# Add a check to ensure we have data to process
if linked_data_processed.shape[0] == 0:
    print("No samples remain after handling missing values. The dataset cannot be processed further.")
    is_trait_biased = True
    unbiased_linked_data = linked_data_processed
else:
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_processed, trait_column)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from retinoblastoma samples, including cell lines, primary tumors, and xenografts."
)

# 6. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Normalized gene data shape: (22, 132)
First few normalized gene symbols: ['C2', 'C3', 'C6', 'C9', 'CX3CR1', 'F10', 'F11', 'F12', 'F2', 'F3']
Normalized gene data saved to ../../output/preprocess/Retinoblastoma/gene_data/GSE29686.csv
Clinical features loaded from ../../output/preprocess/Retinoblastoma/clinical_data/GSE29686.csv
Clinical features shape: (1, 132)
Linked data shape: (132, 23)
First few columns: ['Retinoblastoma', 'C2', 'C3', 'C6', 'C9']
Using trait column: Retinoblastoma
Data type of trait column: float64
Shape after handling missing values: (0, 1)
No samples remain after handling missing values. The dataset cannot be processed further.
Abnormality detected in the cohort: GSE29686. Preprocessing failed.
Data quality check failed. The dataset is not suitable for association studies.
